In [ ]:
!pip install subprocess
!pip install ultralytics
!pip install time
!pip install speedtest

ERROR: Could not find a version that satisfies the requirement subprocess (from versions: none)
ERROR: No matching distribution found for subprocess
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.3/896.3 kB 18.8 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement time (from versions: none)
ERROR: No matching distribution found for time


In [ ]:
!pip install speedtest-cli


In [ ]:
import subprocess
import cv2
from ultralytics import YOLO
import time
import requests

def measure_internet_speed():

    try:

        test_url = "https://speed.hetzner.de/10MB.bin"  # A 10MB file for quick testing


        start_time = time.time()


        response = requests.get(test_url, stream=True, timeout=10, verify=False)

        total_size = 0
        for chunk in response.iter_content(chunk_size=8192):  # 8 KB chunks
            total_size += len(chunk)
            if total_size >= 10 * 1024 * 1024:  # Stop after 10MB
                break

        end_time = time.time()

        # Calculate the time it took to download 10 MB (in seconds)
        download_time = end_time - start_time
        if download_time <= 0:
            print("Error: Download time is too fast or too slow. Speed could not be measured.")
            return None

        # Calculate download speed in Kbps (kilobits per second)
        download_speed_kbps = (total_size * 8) / 1024 / download_time  # Kbps
        print(f"Download Speed: {download_speed_kbps:.2f} Kbps")

        return download_speed_kbps
    except Exception as e:
        print(f"Error measuring internet speed: {e}")
        return None

def get_stream_url(youtube_url, quality='best'):
    """Get the YouTube stream URL at the specified quality."""
    process = subprocess.Popen(
        ['streamlink', youtube_url, quality, '--stream-url'],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    stream_url, error = process.communicate()
    if error:
        raise Exception(f"Error fetching stream URL: {error.decode('utf-8')}")
    return stream_url.decode('utf-8').strip()

def select_quality(speed):
    """Select video quality based on internet speed in Kbps."""
    if speed >= 5000:  # 5000 Kbps = 5 Mbps
        return '720p'
    elif speed >= 2000:  # 2000 Kbps = 2 Mbps
        return '480p'
    else:
        return '360p'

# Define the YouTube live stream URL
youtube_url = 'https://www.youtube.com/live/inDzgZjCxmQ?si=mGfOsMLhGA-SQcA3'

# Load the YOLO model
model = YOLO('/content/drive/MyDrive/behavor detection system predictions/best.pt')

# Main loop to adjust bitrate and process the stream
while True:
    # Measure current internet speed
    speed = measure_internet_speed()
    if speed:
        print(f"Internet speed: {speed:.2f} Kbps")
        # Select quality based on speed
        quality = select_quality(speed)
        print(f"Selected quality: {quality}")

        try:
            # Get the stream URL for the selected quality
            stream_url = get_stream_url(youtube_url, quality)
            print(f"Stream URL: {stream_url}")
        except Exception as e:
            print(f"Error fetching stream: {e}")
            time.sleep(5)
            continue

        # Capture the stream using OpenCV
        cap = cv2.VideoCapture(stream_url)

        if not cap.isOpened():
            print("Error opening video stream")
        else:
            while cap.isOpened():
                ret, frame = cap.read()
                if ret:
                    # Run YOLO predictions
                    results = model(frame)

                    # Count detected behaviors
                    cow_counts = {"eating": 0, "lying": 0, "standing": 0}
                    for result in results:
                        if hasattr(result, 'boxes'):
                            for box in result.boxes:
                                class_id = int(box.cls[0])
                                if class_id == 0:  # Eating
                                    cow_counts["eating"] += 1
                                elif class_id == 1:  # Lying
                                    cow_counts["lying"] += 1
                                elif class_id == 2:  # Standing
                                    cow_counts["standing"] += 1

                    # Print detected counts
                    print(f"Eating: {cow_counts['eating']}, Lying: {cow_counts['lying']}, Standing: {cow_counts['standing']}")

                else:
                    break

        cap.release()
    else:
        print("Failed to measure internet speed. Retrying...")

    # Wait before re-measuring speed
    time.sleep(10)


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'speed.hetzner.de'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Download Speed: 1.95 Kbps
Internet speed: 1.95 Kbps
Selected quality: 360p
Stream URL: https://manifest.googlevideo.com/api/manifest/hls_playlist/expire/1732721175/ei/t-VGZ73QDJf4sfIPis_--A8/ip/34.169.26.140/id/inDzgZjCxmQ.2/itag/93/source/yt_live_broadcast/requiressl/yes/ratebypass/yes/live/1/sgoap/gir%3Dyes%3Bitag%3D140/sgovp/gir%3Dyes%3Bitag%3D134/rqh/1/hdlc/1/hls_chunk_host/rr3---sn-qxoedn7k.googlevideo.com/xpc/EgVo2aDSNQ%3D%3D/playlist_duration/30/manifest_duration/30/spc/qtApAVE8tZGEFeh0Sohy89anI4X6tP_H_tYvzMJObVRCGgL-IttfX8RmIiLNb6Q/vprv/1/playlist_type/DVR/initcwndbps/20843750/met/1732699575,/mh/CV/mm/44/mn/sn-qxoedn7k/ms/lva/mv/m/mvi/3/pl/17/rms/lva,lva/dover/11/pacing/0/keepalive/yes/fexp/51319289,51326932,51335594,51355912/mt/1732699237/sparams/expire,ei,ip,id,itag,source,requiressl,ratebypass,live,sgoap,sgovp,rqh,hdlc,xpc,playlist_duration,manifest_duration,spc,vprv,playlist_type/sig/AJfQdSswRgIhAPUZzU04tdz2CIocKBw-NjsYqiAWyDPWN7kj7Yiv_dpWAiEA7kZ0rPoKrASCuJrngqfe_yOL8Z22oGe

KeyboardInterrupt: 